<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/Vers%C3%A3o_2_(Valida%C3%A7%C3%A3o_cruzada).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Machine learning for predicting liver and/or lung metastasis in colorectal cancer: A retrospective study based on the SEER database***

Este estudo propõe um modelo de aprendizado de máquina para prever o risco de metástase hepática e/ou pulmonar em pacientes com câncer colorretal (CRC). A partir da base de dados SEER, foram extraídos dados aproximadamente 53 mil pacientes com diagnóstico patológico de CRC entre 2010 e 2015, desenvolvendo sete modelos de algoritmos(Decision tree, Randon Forest, Naive Bayes,  KNN,XGBoost, Gradient Boosting.

### Parte 1:  Importação das Bibliotecas e Carregamento do Dataset

Nesta etapa, importamos as bibliotecas necessárias para análise e carregamos o dataset. Realizamos uma verificação inicial para identificar e remover valores faltantes e definimos as variáveis preditoras (X) e as variáveis alvo (y), preparando os dados para o pré-processamento e a modelagem.

In [7]:
# Importação das bibliotecas essenciais para análise de dados, visualização e aprendizado de máquina
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importação de funções e classes específicas para pré-processamento, modelagem e avaliação
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix,
                             classification_report, precision_recall_curve, average_precision_score)
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/dataset/refs/heads/main/export.csv')

# Verificar se existem valores faltantes (NaN) e exibir quantos valores faltantes existem por coluna
print("Valores faltantes por coluna:\n", df.isnull().sum())

# Remover as linhas com valores faltantes
df.dropna(inplace=True)

# Definir as variáveis preditoras (X)
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis',
        'Origin recode NHIA (Hispanic, Non-Hisp)']]

# Definir as variáveis alvo: Metástase no fígado e no pulmão
y_liver = df['SEER Combined Mets at DX-liver (2010+)']
y_lung = df['SEER Combined Mets at DX-lung (2010+)']



Valores faltantes por coluna:
 Patient ID                                         0
Age recode with <1 year olds                       0
Sex                                                0
Race recode (White, Black, Other)                  0
Histologic Type ICD-O-3                            0
Grade Recode (thru 2017)                           0
Primary Site                                       0
Derived AJCC T, 7th ed (2010-2015)                 0
Derived AJCC N, 7th ed (2010-2015)                 0
CS tumor size (2004-2015)                          0
CEA Pretreatment Interpretation Recode (2010+)     0
Tumor Deposits Recode (2010+)                      0
Marital status at diagnosis                        0
Origin recode NHIA (Hispanic, Non-Hisp)            0
SEER Combined Mets at DX-lung (2010+)             15
SEER Combined Mets at DX-liver (2010+)            12
SEER Combined Mets at DX-bone (2010+)             14
ICD-O-3 Hist/behav                                 0
ICD-O-3 Hist/be

###Parte 2:  Preparação das Variáveis Alvo e Codificação de Variáveis Categóricas

Nesta etapa, preparamos as variáveis alvo (y), combinando as informações de metástase hepática e pulmonar em uma coluna binária para indicar a presença de metástase. Também aplicamos LabelEncoder para transformar variáveis categóricas de X em valores numéricos, facilitando o uso dos dados em modelos de aprendizado de máquina.

In [8]:
# Concatenar as variáveis alvo 'y_liver' e 'y_lung' em um DataFrame 'y' para problemas multi-label
y = pd.concat([y_liver, y_lung], axis=1)

# Aplicar codificação a variáveis categóricas em 'X' usando LabelEncoder, para prepará-las para o modelo
for col in X.columns:
    if X[col].dtype == 'object':  # Verifica se a coluna é categórica (strings)
        X[col] = LabelEncoder().fit_transform(X[col])

# Função para combinar as informações de metástase hepática e pulmonar em uma coluna binária 'Binary Mets'
# (0 = sem metástase, 1 = com metástase)
def combine_mets_binary(row):
    if row['SEER Combined Mets at DX-liver (2010+)'] == 'Yes' or row['SEER Combined Mets at DX-lung (2010+)'] == 'Yes':
        return 1  # Com metástase
    else:
        return 0  # Sem metástase

# Aplicar a função para criar a nova coluna binária 'Binary Mets' em 'y'
y['Binary Mets'] = y.apply(combine_mets_binary, axis=1)

# Verificar se 'X' e 'y' têm o mesmo número de amostras
print(f"Tamanho de X: {len(X)}")
print(f"Tamanho de y: {len(y)}")

# Salvar o DataFrame 'y' em um arquivo CSV para referência futura ou análise adicional
y.to_csv('/content/Y.csv')

<ipython-input-8-1cd8977feba9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-8-1cd8977feba9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-8-1cd8977feba9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Tamanho de X: 53448
Tamanho de y: 53448


###Parte 3: Definição e Configuração dos Modelos de Aprendizado de Máquina e Validação Cruzada

Aqui, configuramos os principais algoritmos de aprendizado de máquina, incluindo Decision Tree, Random Forest, SVM, Naive Bayes, KNN, XGBoost e Gradient Boosting. Cada modelo é definido com parâmetros específicos para otimizar o desempenho. Em seguida, aplicamos uma validação cruzada estratificada com 5 divisões para avaliar e comparar a performance dos modelos de maneira consistente e robusta.

In [9]:
# Definição dos modelos de aprendizado de máquina com hiperparâmetros ajustados
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(),
    "XGBoost": XGBClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}
# Configuração da validação cruzada estratificada com 5 divisões (folds)
# Isso garante que a proporção de classes seja mantida em cada divisão, e o shuffle embaralha os dados antes de dividir
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### Parte 4: Avaliação e Comparação dos Modelos de Aprendizado de Máquina em Conjuntos de Treino, Validação e Teste


Este bloco de código implementa a validação cruzada para treinar e avaliar os modelos de aprendizado de máquina definidos no pipeline. Ele utiliza a técnica de K-Fold Cross-Validation para dividir os dados em múltiplos folds, garantindo uma avaliação robusta do desempenho dos modelos. Durante cada fold, os dados de treinamento são balanceados utilizando o SMOTE e escalados com o StandardScaler. Métricas de desempenho, como precisão, recall, F1-Score, especificidade, AUC-ROC e AUPR, são calculadas tanto para o conjunto de treinamento quanto para o conjunto de teste. Além disso, visualizações como matrizes de confusão e curvas ROC e Precisão-Recall são geradas. Ao final, as métricas médias de todos os folds são compiladas para comparação.

In [12]:
results = []  # Lista para armazenar os resultados das métricas de cada modelo

# Iterando sobre todos os modelos no dicionário 'models'
for model_name, model in models.items():
    print(f"\nProcessando o modelo: {model_name}")

    # Inicializando acumuladores para métricas médias
    avg_metrics = {
        "precision_train": [],
        "recall_train": [],
        "f1_train": [],
        "specificity_train": [],
        "auc_pr_train": [],
        "auc_score_train": [],
        "precision_test": [],
        "recall_test": [],
        "f1_test": [],
        "specificity_test": [],
        "auc_pr_test": [],
        "auc_score_test": []
    }

    fold_number = 1  # Contador de folds

    # Validação cruzada
    for train_index, test_index in kf.split(X, y['Binary Mets']):
        print(f"\nFold {fold_number}:")
        print(f"  Índices de treinamento: {train_index[:10]}... (Total: {len(train_index)})")
        print(f"  Índices de teste: {test_index[:10]}... (Total: {len(test_index)})")

        # Dividindo os dados
        X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
        y_train_cv, y_test_cv = y['Binary Mets'].iloc[train_index], y['Binary Mets'].iloc[test_index]

        # Aplicar SMOTE no conjunto de treinamento
        smote = SMOTE(random_state=42)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train_cv, y_train_cv)
        print(f"  Aplicado SMOTE no fold {fold_number}: Conjunto de treinamento balanceado.")

        # Escalar os dados após o balanceamento
        scaler = StandardScaler()
        X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)
        X_test_scaled = scaler.transform(X_test_cv)  # Escalar o conjunto de teste com os parâmetros do treino

        # Treinando o modelo
        model.fit(X_train_balanced_scaled, y_train_balanced)
        print(f"  Modelo ajustado no fold {fold_number}.")

        # Previsões no treino
        y_train_pred = model.predict(X_train_balanced_scaled)
        y_train_pred_proba = model.predict_proba(X_train_balanced_scaled)[:, 1]

        # Previsões no teste
        y_test_pred = model.predict(X_test_scaled)
        y_test_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

        # Métricas no treino
        cm_train = confusion_matrix(y_train_balanced, y_train_pred)
        tn_train, fp_train, fn_train, tp_train = cm_train.ravel()
        specificity_train = tn_train / (tn_train + fp_train)
        precision_train = tp_train / (tp_train + fp_train)
        recall_train = tp_train / (tp_train + fn_train)
        f1_train = f1_score(y_train_balanced, y_train_pred)
        auc_pr_train = average_precision_score(y_train_balanced, y_train_pred_proba)
        auc_score_train = roc_auc_score(y_train_balanced, y_train_pred_proba)

        print(f"  Métricas no treino do fold {fold_number}:")
        print(f"    Precisão: {precision_train:.4f}, Recall: {recall_train:.4f}, F1-Score: {f1_train:.4f}")
        print(f"    Especificidade: {specificity_train:.4f}, AUC-ROC: {auc_score_train:.4f}, AUPR: {auc_pr_train:.4f}")

        # Salvando métricas do treino
        avg_metrics["specificity_train"].append(specificity_train)
        avg_metrics["precision_train"].append(precision_train)
        avg_metrics["recall_train"].append(recall_train)
        avg_metrics["f1_train"].append(f1_train)
        avg_metrics["auc_pr_train"].append(auc_pr_train)
        avg_metrics["auc_score_train"].append(auc_score_train)

        # Métricas no teste
        cm_test = confusion_matrix(y_test_cv, y_test_pred)
        tn_test, fp_test, fn_test, tp_test = cm_test.ravel()
        specificity_test = tn_test / (tn_test + fp_test)
        precision_test = tp_test / (tp_test + fp_test)
        recall_test = tp_test / (tp_test + fn_test)
        f1_test = f1_score(y_test_cv, y_test_pred)
        auc_pr_test = average_precision_score(y_test_cv, y_test_pred_proba)
        auc_score_test = roc_auc_score(y_test_cv, y_test_pred_proba)

        print(f"  Métricas no teste do fold {fold_number}:")
        print(f"    Precisão: {precision_test:.4f}, Recall: {recall_test:.4f}, F1-Score: {f1_test:.4f}")
        print(f"    Especificidade: {specificity_test:.4f}, AUC-ROC: {auc_score_test:.4f}, AUPR: {auc_pr_test:.4f}")

        # Salvando métricas do teste
        avg_metrics["specificity_test"].append(specificity_test)
        avg_metrics["precision_test"].append(precision_test)
        avg_metrics["recall_test"].append(recall_test)
        avg_metrics["f1_test"].append(f1_test)
        avg_metrics["auc_pr_test"].append(auc_pr_test)
        avg_metrics["auc_score_test"].append(auc_score_test)

        fold_number += 1

    # Adicionando métricas médias no resultado final
    results.append({
        "Modelo": model_name,
        "Precision (Treino)": np.mean(avg_metrics["precision_train"]),
        "Recall (Treino)": np.mean(avg_metrics["recall_train"]),
        "F1-Score (Treino)": np.mean(avg_metrics["f1_train"]),
        "Especificidade (Treino)": np.mean(avg_metrics["specificity_train"]),
        "AUC (ROC - Treino)": np.mean(avg_metrics["auc_score_train"]),
        "AUPR (Treino)": np.mean(avg_metrics["auc_pr_train"]),
        "Precision (Teste)": np.mean(avg_metrics["precision_test"]),
        "Recall (Teste)": np.mean(avg_metrics["recall_test"]),
        "F1-Score (Teste)": np.mean(avg_metrics["f1_test"]),
        "Especificidade (Teste)": np.mean(avg_metrics["specificity_test"]),
        "AUC (ROC - Teste)": np.mean(avg_metrics["auc_score_test"]),
        "AUPR (Teste)": np.mean(avg_metrics["auc_pr_test"])
    })

# **EXIBINDO OS RESULTADOS FINAIS EM FORMATO DE TABELA**
print("\nTabela Comparativa de Resultados (Treinamento vs. Teste):")
results_df = pd.DataFrame(results)
print(results_df)


Processando o modelo: Decision Tree

Fold 1:
  Índices de treinamento: [ 0  2  3  4  5  6  7  8  9 10]... (Total: 42758)
  Índices de teste: [ 1 12 17 21 23 24 25 26 30 33]... (Total: 10690)
  Aplicado SMOTE no fold 1: Conjunto de treinamento balanceado.
  Modelo ajustado no fold 1.
  Métricas no treino do fold 1:
    Precisão: 0.9982, Recall: 0.9975, F1-Score: 0.9978
    Especificidade: 0.9982, AUC-ROC: 1.0000, AUPR: 1.0000
  Métricas no teste do fold 1:
    Precisão: 0.3560, Recall: 0.5332, F1-Score: 0.4269
    Especificidade: 0.8326, AUC-ROC: 0.6825, AUPR: 0.2590

Fold 2:
  Índices de treinamento: [ 0  1  3  5  7  8  9 10 11 12]... (Total: 42758)
  Índices de teste: [ 2  4  6 13 14 19 35 38 40 51]... (Total: 10690)
  Aplicado SMOTE no fold 2: Conjunto de treinamento balanceado.
  Modelo ajustado no fold 2.
  Métricas no treino do fold 2:
    Precisão: 0.9979, Recall: 0.9976, F1-Score: 0.9977
    Especificidade: 0.9979, AUC-ROC: 1.0000, AUPR: 1.0000
  Métricas no teste do fold 2:
  